In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [15]:
# ROI metrics
def calculate_sortino_ratio(daily_returns, risk_free_rate=0.01):
    negative_returns = daily_returns[daily_returns < 0]
    downside_deviatiaon = np.std(negative_returns, ddof=1)
    excess_daily_returns = daily_returns - risk_free_rate / 365
    sortino_ratio = np.mean(excess_daily_returns) / downside_deviatiaon
    return sortino_ratio * np.sqrt(252)  # annualised

def calculate_sharpe_ratio(daily_returns, risk_free_rate=0.01):
    excess_daily_returns = daily_returns - risk_free_rate / 365
    sharpe_ratio = np.mean(excess_daily_returns) / np.std(excess_daily_returns, ddof=1)
    return sharpe_ratio * np.sqrt(252)  # annualised

def calculate_maximum_drawdown(cumulative_returns):
    cumulative_returns = np.array(cumulative_returns) + 1
    if len(cumulative_returns) > 0:
        running_max = np.maximum.accumulate(cumulative_returns)
        drawdowns = (running_max - cumulative_returns) / running_max
        max_drawdown = np.max(drawdowns)
        return max_drawdown
    else:
        return 0

def calculate_calmar_ratio(annual_return, max_drawdown):
    # if max_drawdown is negative, make it positive for the ratio
    return annual_return / abs(max_drawdown)

In [16]:
# test set for testing
test = pd.read_csv('/min_test.csv', index_col=0)


test.head()

# train set for standardising
df_train = pd.read_csv('/min_train.csv', index_col=0)

In [17]:
mp_tr = df_train['Mid_Price']
s_tr = df_train['Bid_Ask_Spread']

In [18]:
from tensorflow.keras.models import load_model

lstm_mp = load_model('/mp_LSTM.h5', compile=False)
lstm_bas = load_model('/spread_LSTM.h5', compile=False)

In [19]:
# simulation for LSTM
def LSTM_trading_simulation(future_buy_increase, future_sell_decrease, quantile_optimised):
    # Initial conditions
    initial_cash = 10000  # Starting cash
    cash = initial_cash
    shares_owned = 0  # Starting shares owned
    transactions = []  # List to record transactions
    daily_portfolio_values = []  # Track daily portfolio values for performance metrics calculation
    buy_signals = 0
    sell_signals = 0

    # Parameters for market impact, slippage, and fees
    market_impact_constant = 0.1
    average_daily_volume = test['Mid_Price'].count()  # Using total count as a proxy
    brokerage_fee_bps = 0.005
    exchange_fee_per_share = 0.003
    shares_per_transaction = 500

    look_ahead_window = 5
    look_back_window = 20

    for i in range(look_back_window, len(test) - look_ahead_window):
      # Ask1 and bid 1
      #current_ask_price = test.iloc[i]['Ask1_Price']
      #current_bid_price = test.iloc[i]['Bid1_Price']


      # Mid_price
      current_mid_price = test.iloc[i]['Mid_Price']
      # predictive avg mid price
      # (1) find the past 20 mid price
      # simulation for LSTM
      x_mp = test.iloc[i-look_back_window:i]['Mid_Price'].to_numpy()
      x_mp = x_mp.reshape(1, -1, 1)  # Corrected reshaping
      x_mp = (x_mp - np.mean(mp_tr)) / np.std(mp_tr)

      y_mp = lstm_mp.predict(x_mp) # result is a (1, 5, 1) array that has been scaled
      # unscale the prediction values
      y_mp_unscaled = y_mp*np.std(mp_tr)+np.mean(mp_tr)
      # reshape to (1, 5)
      y_mp_unscaled = y_mp_unscaled.reshape(-1,y_mp_unscaled.shape[1])

      future_avg_price = y_mp_unscaled.mean()

      #future_prices = data_test.iloc[i+1:i+look_ahead_window+1]['Mid-Price']


      # Bid_ask_spread

      #past bas
      past_bas = test.iloc[i-look_back_window:i]['Bid_Ask_Spread']
      bid_ask_spread_threshold = past_bas.quantile(quantile_optimised)
      # predictive bid ask
      # (1) past bas
      x_bas = test.iloc[i-look_back_window:i]['Bid_Ask_Spread'].to_numpy()
      x_bas = x_bas.reshape(1, -1, 1)  # Corrected reshaping
      x_bas = (x_bas - np.mean(s_tr)) / np.std(s_tr)
        # predcition
      y_bas = lstm_bas.predict(x_bas)
      # unscale the prediction values
      y_bas_unscaled = y_bas*np.std(s_tr)+np.mean(s_tr)
      # reshape to (1, 5)
      y_bas_unscaled = y_bas_unscaled.reshape(-1,y_mp_unscaled.shape[1])
      #future_bid_ask = data_test.iloc[i+1:i+look_ahead_window+1]['Bid_Ask_Spread']
      future_bid_ask_avg = y_bas_unscaled.mean()
      is_liquid = future_bid_ask_avg <= bid_ask_spread_threshold



      if future_avg_price > current_mid_price * future_buy_increase and is_liquid and cash > 0:
          # Buy condition
          shares_to_buy = min(shares_per_transaction, int(cash / (current_mid_price + brokerage_fee_bps * current_mid_price + exchange_fee_per_share)))
          volume_ratio = (shares_to_buy) / average_daily_volume
          price_impact = market_impact_constant * (volume_ratio ** 0.5) * current_mid_price
          adjusted_buy_price = current_mid_price + price_impact
          total_cost = adjusted_buy_price * shares_to_buy + brokerage_fee_bps * adjusted_buy_price * shares_to_buy + exchange_fee_per_share * shares_to_buy
          cash -= total_cost
          shares_owned += shares_to_buy
          transactions.append(('Buy', shares_to_buy, adjusted_buy_price, total_cost, cash, i))
          buy_signals += 1

      elif future_avg_price < current_mid_price * future_sell_decrease and is_liquid and shares_owned > 0:
          # Sell condition
          shares_to_sell = min(shares_per_transaction, shares_owned)
          volume_ratio = shares_to_sell / average_daily_volume
          price_impact = market_impact_constant * (volume_ratio ** 0.5) * current_mid_price
          adjusted_sell_price = current_mid_price - price_impact
          total_revenue = adjusted_sell_price * shares_to_sell - brokerage_fee_bps * adjusted_sell_price * shares_to_sell - exchange_fee_per_share * shares_to_sell
          cash += total_revenue
          transactions.append(('Sell', shares_to_sell, adjusted_sell_price, total_revenue, cash, i))
          shares_owned = 0
          sell_signals += 1

      # Hold condition
      else:
          transactions.append(('Hold', shares_owned, current_mid_price, 0, cash, i))

      # Update the portfolio value for each day
      portfolio_value = cash + shares_owned * current_mid_price
      daily_portfolio_values.append(portfolio_value)

    # Performance Metrics
    daily_returns = pd.Series(daily_portfolio_values).pct_change().dropna() # daily returns from portfolio values
    sortino_ratio = calculate_sortino_ratio(daily_returns)
    sharpe_ratio = calculate_sharpe_ratio(daily_returns)
    cumulative_returns = (1 + daily_returns).cumprod() - 1
    max_drawdown = calculate_maximum_drawdown(cumulative_returns)
    if len(test) > 0 and (cumulative_returns.iloc[-1] + 1) > 0:
      annual_return = (cumulative_returns.iloc[-1] + 1) ** (365 / len(test)) - 1
    else:
      # cases where annual return cannot be computed
      annual_return = None
    calmar_ratio = calculate_calmar_ratio(annual_return, max_drawdown)

    # Return the overall profit or loss for this simulation run
    final_valuation = cash + shares_owned * test.iloc[-1]['Mid_Price']
    return initial_cash, final_valuation, final_valuation - initial_cash, transactions, buy_signals, sell_signals, sortino_ratio, sharpe_ratio, max_drawdown, calmar_ratio, daily_returns

In [20]:

# Define your parameters
future_buy_increase = 1.01
future_sell_decrease = 0.97
quantile_optimised = 0.85

# Run simulation
try:
    results = LSTM_trading_simulation(future_buy_increase, future_sell_decrease, quantile_optimised)
    if len(results) != 11:
        raise ValueError("Expected 11 outputs from LSTM_trading_simulation, received {}".format(len(results)))

    (initial_cash_test, final_valuation_test, profit_loss_test, transactions_test, buy_signals_test,
     sell_signals_test, sortino_ratio_test, sharpe_ratio_test, max_drawdown_test, calmar_ratio_test, daily_returns_test) = results
except Exception as e:
    print("Error in simulation or unpacking results:", e)
    raise

# Initialize the 'test_result' DataFrame with default 'Hold' signals
test_result = pd.DataFrame()  # Create an empty DataFrame
test_result['Trading Signal'] = 'Hold'

# Update the DataFrame with transactions for training of RF
for transaction in transactions_test:
    signal_type, shares, price, cost, cash, day_index = transaction
    test_result.at[day_index, 'Trading Signal'] = signal_type

# Collecting results
results_list = []
results_list.append((future_buy_increase, future_sell_decrease, quantile_optimised, profit_loss_test, len(transactions_test), buy_signals_test, sell_signals_test, sortino_ratio_test, sharpe_ratio_test, max_drawdown_test, calmar_ratio_test))

# Create DataFrame from results
results_df = pd.DataFrame(results_list, columns=['Future Buy Increase', 'Future Sell Decrease', 'Quantile Optimised', 'Profit/Loss', 'Number of Transactions', 'Number of Buy Signals', 'Number of Sell Signals', 'Sortino Ratio', 'Sharpe Ratio', 'Max Drawdown', 'Calmar Ratio'])

# Display the first 50 rows of the results DataFrame
results_df.head(50)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 34ms/step


,Future Buy Increase,Future Sell Decrease,Quantile Optimised,Profit/Loss,Number of Transactions,Number of Buy Signals,Number of Sell Signals,Sortino Ratio,Sharpe Ratio,Max Drawdown,Calmar Ratio
0,1.01,0.97,0.85,70334.221434,10685,5325,1030,0.984596,0.553144,0.470325,0.159401


In [21]:
test_result

,Trading Signal
20,Buy
21,Hold
22,Sell
23,Hold
24,Buy
...,...
10700,Hold
10701,Hold
10702,Sell
10703,Buy


In [23]:
daily_returns_test

1        0.005034
2        0.158694
3        0.000000
4       -0.014158
5        0.119238
           ...   
10680   -0.133003
10681    0.194911
10682   -0.081391
10683   -0.022372
10684    0.116053
Length: 10684, dtype: float64

In [22]:
daily_returns_test
cumulative_returns_test = (1 + daily_returns_test).cumprod() - 1
cumulative_returns_test

1        0.005034
2        0.164527
3        0.164527
4        0.148039
5        0.284928
           ...   
10680    5.965042
10681    7.322605
10682    6.645218
10683    6.474182
10684    7.341585
Length: 10684, dtype: float64

In [24]:
# export important df
test_result.to_csv('LSTM_signal_s1.csv')
results_df.to_csv('LSTM_results_s1.csv')